# 三维高斯喷溅 3D Gaussian Splatting 

- Reference: [【较真系列】讲人话-3d gaussian splatting全解(原理+代码+公式)](https://space.bilibili.com/644569334/channel/collectiondetail?sid=3173291&spm_id_from=333.788.0.0)

## 1. 3D Gaussian：概率论，椭球

- 什么是 splatting？
  - 一种体渲染（Volumetric Rendering）的方法，把三维物体渲染到二维平面上
    - Ray Casting 是 *被动* 的
      - 计算出每个 像素点 受到的 发光粒子 的影响
      - NERF 中用的体渲染算法
    - Splatting 是 *主动* 的
      - 计算出每个 发光粒子 如何影响到 像素点
      - 1990 年的老东西了，快，但效果不好
  - splat，拟声词，bia唧一声
    - 把雪球扔到墙上，留下一个引子，这个印子称作 footprint
  - Splatting 的核心：
    - 选择雪球
    - 抛雪球，得到 footprint
    - 对 footprint 进行 blending，生成图象
- 为什么选择 3D Gaussian 作为雪球？
  - 很好的数学性质：仿射变换后高斯核依旧闭合
    - 仿射变换 等价于 齐次坐标下的 transformation
      - $\mathbf{w} = \mathbf{A} \mathbf{x} + \mathbf{b}$
    - 3D Gaussian 降维到二维之后，依然是 2D Gaussian
      - 这里的降维在数学上体现为：沿着某个坐标轴积分

- 3D Gaussian
  - 椭球高斯：$\displaystyle G(\mathbf{x}) = \dfrac{1}{\sqrt{(2 \pi)^k |\mathbf{\Sigma}|}} \exp \left( -\dfrac{1}{2} (\mathbf{x} - \mathbf{\mu})^{\top} \mathbf{\Sigma}^{-1} (\mathbf{x} - \mathbf{\mu}) \right)$
  - $\mathbf{\mu}$ 是均值
  - $\mathbf{\Sigma}$ 是 协方差矩阵，对称，半正定，$|\mathbf{\Sigma}|$ 为其行列式
- 3D Gaussian 为什么是椭球？
  - 一句话：概率密度的 Level Set （即等概率密度面）是椭球面
  - 3D Gaussian：PDF 的值为常数，那么 exp 的指数为常数，展开就是平移、缩放和旋转过的 unit sphere
  - 各项同性：标准球；各项异性：椭球
- 协方差矩阵是如何控制椭球形状的？
  - 任何高斯分布都可以视作标准高斯分布通过仿射变换得到的
  - 高斯分布：
    - $\mathbf{x} \in \mathbb{R}^3 \sim \mathcal{N}(\mathbf{\mu}, \mathbf{\Sigma})$
    - 均值 $(\mu_1, \mu_2, \mu_3)$
    - 协方差矩阵 $\mathbf{\Sigma}$
  - 高斯分布的仿射变换：
    - $\mathbf{w} = \mathbf{A} \mathbf{x} + \mathbf{b}$
    - $\mathbf{w} \sim \mathcal{N}(\mathbf{A} \mathbf{\mu} + \mathbf{b}, \mathbf{A} \mathbf{\Sigma} \mathbf{A}^{\top})$
    - 即 $\mathbf{\Sigma} = \mathbf{A} \ \mathbf{I} \ \mathbf{A}^{\top}$
- 协方差矩阵为什么能用旋转和缩放矩阵表达出来？
  - $\mathbf{A} = \mathbf{R} \ \mathbf{S}$，则 $\mathbf{\Sigma} = \mathbf{A} \ \mathbf{I} \ \mathbf{A}^{\top} = \mathbf{R} \ \mathbf{S} \ \mathbf{S}^{\top} \ \mathbf{R}^{\top}$
  - 几何变换角度的解释：
    - 仿射变换包括旋转、缩放和平移
    - 平移对应 $\mathbf{b}$，那么旋转和缩放必然对应 $\mathbf{A}$
  - 线性代数角度的解释：
    - 协方差矩阵对称且半正定
    - 这样的矩阵可以被特征值分解为 $\mathbf{\Sigma} = \mathbf{Q} \ \mathbf{\Lambda} \ \mathbf{Q}^{\top}$，其中：
      - $\mathbf{Q}$ 为正交矩阵，对应旋转矩阵；
      - $\mathbf{\Lambda}$ 为对角矩阵，对应缩放矩阵和他自己的转置相乘。
- `computeCov3D` 这个 CUDA kernel 就是在用旋转和缩放矩阵来算协方差矩阵
  - `__device__ void computeCov3D(const glm::vec3 scale, float mod, const glm::vec4 rot, float * cov3D);`

## 2. 3D Gaussian：渲染流水线

- 相机模型：连接 3D 世界与 2D 图片
  - 坐标系
    - ![坐标系](https://learnopengl.com/img/getting-started/coordinate_systems.png)
    - 模型坐标系（Model-space Coordinate，Local Coordinate）
    - 世界坐标系（World-space Coordinate）
    - 相机坐标系（Camera-space Coordinate，View-space Coordinate）
    - 归一化相机坐标系（Normalized Device Coordinate，NDC，Clip-space Coordinate）
    - 像素坐标系（Screen-space Coordinate）
    - $\mathbf{V}_{\mathrm{clip}} \in [-1, 1]^3 = \mathbf{P} \, \mathbf{V} \,  \mathbf{M} \, \left( \mathbf{V}_{\mathrm{local}} \in \mathbb{R}^3 \right)$
  - 视锥和坐标变换
    - ![视锥](./view_frustum.jpg)
    - Perspective projection is to *stretch* the view frustum into a canonical view volume. 
    - The common practive is to map view-space coordinate $(x, y, z, 1)^\top$ into NDC $(x^{'}, y^{'}, z^{'}, z)^\top = (x_p, y_p, z_p, 1)^\top$.
      - For $x$ and $y$: 
        - The near plane is bounded by $(l, r, b, t)$ where $z = n$;
        - An internal slice inside the frustum where $z = z$ should be bounded by $\left( \dfrac{z}{n} l, \dfrac{z}{n} r, \dfrac{z}{n} b, \dfrac{z}{n} t \right)$;
        - This internal slice should be stretched into $[-1, 1, -1, 1]$;
        - This yields the first two lines in the following equation.
          - Note that the coefficients for $z$ are handled by the $w$ dimension in the actual projection matrix. 
      - For $z$:
        - We need to map $z$ from $[n, f]$ into $[-1, 1]$.
        - This yields the last three lines in the following equation. 
    - 那么：
    $
    \left\{
    \begin{aligned}
    \dfrac{x - \dfrac{z}{n} l }{r - l} & = \dfrac{x_p + 1}{2} \\
    \dfrac{y - \dfrac{z}{n} b }{t - b} & = \dfrac{y_p + 1}{2} \\
    z_p & = \dfrac{A z + B}{z} \\
    z_p(n) & = -1 \\
    z_p(f) & = 1
    \end{aligned}
    \right.
    ;
    $
    - 解得投影变换矩阵如下：
    $
    \mathbf{P} = 
    \begin{pmatrix}
       \dfrac{2n}{r - l} & 0 & -\dfrac{r + l}{r - l} & 0 \\
       0 & \dfrac{2n}{t - b} & -\dfrac{t + b}{t - b} & 0 \\ 
       0 & 0 & -\dfrac{n + f}{n - f} & \dfrac{2nf}{n - f} \\ 
       0 & 0 & 1 & 0
    \end{pmatrix}
    $
    - 特别地，如果视锥是对称的（$-l = r = \dfrac{w}{2}, -b = t = \dfrac{h}{2}$），则
    $
    \mathbf{P} = 
    \begin{pmatrix}
       \dfrac{2n}{w} & 0 & 0 & 0 \\
       0 & \dfrac{2n}{h} & 0 & 0 \\ 
       0 & 0 & -\dfrac{n + f}{n - f} & \dfrac{2nf}{n - f} \\ 
       0 & 0 & 1 & 0
    \end{pmatrix}
    $  

- 3D Gaussian 的 观测变换
  - 世界坐标系：
    - 高斯核中心 $t_k = (t_0, t_1, t_2)^{\top}$
    - 高斯核：$r_k^{''}(t) = G_{V_k^{''}}(t - t_k)$
      - $V_k^{''}$ 是协方差矩阵
  - 相机坐标系：
    - 高斯核中心 $u_k = (u_0, u_1, u_2)^{\top}$
    - 高斯核：$r_k^{'}(u) = G_{V_k^{'}}(u - u_k)$
    - 均值 $u_k = \mathrm{affine}(t_k) = W t_k + d$ 
    - 协方差矩阵 $V_k^{'} = W \  V_k^{''} \ W^{\top}$

- 3D Gaussian 的 投影变换
  - 相机坐标系：
    - 高斯核中心 $u_k = (u_0, u_1, u_2)^{\top}$
    - $V_k^{'}$ 是协方差矩阵
  - 投影变换：
    - 高斯核中心 $x_k = (x_0, x_1, x_2)^{\top}$
    - 高斯核：$r_k^(x) = G_{V_k}(x - x_k)$
    - 均值 $x_k = m(u_k)$，$m$ 为投影变换，**非线性**
    - 协方差矩阵 $V_k = J \  V_k^{'} \ J^{\top}$，用 $m$ 的 Jacobian 来近似
      - 论文里的公式：$V_k = J \ W \  V_k^{''} \ W^{\top} \ J^{\top}$，又把 $ V_k^{'}$ 给代换了一次
      - Jacobian 
        - $\mathbf{f}(x, y) = (f_1(x, y), f_2(x, y))^{\top}  = (x + \sin(y), y + \sin(x))^{\top}$
        - $\mathbf{J} = \dfrac{\partial (f_1, f_2)}{\partial (x, y)} = \begin{pmatrix} \dfrac{\partial f_1}{\partial x}& \dfrac{\partial f_1}{\partial y} \\ \dfrac{\partial f_2}{\partial x}& \dfrac{\partial f_2}{\partial y} \end{pmatrix} = \begin{pmatrix} 1 & \cos(y) \\ \cos(x) & 1 \end{pmatrix}$
      - 具体到 投影变换 的 Jacobian：
        - $\mathbf{J} = \begin{pmatrix} \dfrac{n}{z} & 0 & -\dfrac{nx}{z^2} \\ 0 & \dfrac{n}{z} & -\dfrac{ny}{z^2} \\ 0 & 0 & -\dfrac{nf}{z^2} \end{pmatrix}$
  - 此时均值和协方差在同一个坐标系里面吗？
    - 不在！
    - 均值：在 NDC 里，范围 $[-1, 1]^3$
    - 协方差：在 **未缩放** 的正交坐标系里，范围 $[l, r] \times [b, t] \times [f, n]$

- 3D Gaussian 的 视口变换
  - 投影变换后：
    - 高斯核中心 $x_k = (x_0, x_1, x_2)^{\top}$
    - 高斯核：$r_k^(x) = G_{V_k}(x - x_k)$
  - 像素坐标系：
    - 高斯核中心 $\mu_k = (\mu_0, \mu_1, \mu_2)^{\top}$
      - 平移 + 缩放
    - 协方差：？
    - 足迹渲染：离散计算
      - $G(x) = \exp \left(-\dfrac{1}{2} (\mathbf{x} - \mathbf{\mu})^{\top} {\mathbf{V}_k}^{-1} (\mathbf{x} - \mathbf{\mu}) \right)$

## 3. 3D Gaussian 球的颜色

- 基函数
  - Fourier 级数收敛到自己的函数可以分解成 sin 和 cos（基函数）的线性组合
  - $\displaystyle f(x) = a_0 + \sum_{n=1}^{\infty} a_n \cos \dfrac{n \pi}{l} x + \sum_{n=1}^{\infty} b_n \sin \dfrac{n \pi}{l} x$
- 球谐函数
  - 任何一个球面坐标的函数可以用多个球谐函数来近似
  - $\displaystyle f(t) = \sum_l \sum_{m=-l}^l c_l^m y_l^m(\theta, \phi)$
  - 其中 $l$ 是阶数，$c_l^m$ 是各项系数，值是 RGB 向量；$y_l^m$ 是基函数，值是标量
  - 其实就是把 Fourier 展开给换元了
  - 举例：3 阶球谐函数总共有 16 项求和：0 阶 1 项，1 阶 3 项，2 阶 5 项，3 阶 7 项
    $$
    \begin{aligned}
    f(t) = \phantom{ }
    & c_0^0 y_0^0 + \\
    & c_1^{-1} y_1^{-1} + c_1^{0} y_1^{0} + c_1^{1} y_1^{1} + \\
    & c_2^{-2} y_2^{-2} + c_2^{-1} y_2^{-1} + c_2^{0} y_2^{0} + c_2^{1} y_2^{1} + c_2^{2} y_2^{2} + \\
    & c_3^{-3} y_3^{-3} + c_3^{-2} y_3^{-2} + c_3^{-1} y_3^{-1} + c_3^{0} y_3^{0} + c_3^{1} y_1^{1} + c_3^{2} y_3^{2} + c_3^{3} y_3^{3} \\
    \end{aligned}
    $$
  - 上式中 0 阶的 $y$ 和方向无关，更高阶的 $y$ 通过方向控制 RGB
  $$
  \left\{
  \begin{aligned}
  y_0^0 & = \sqrt{\dfrac{1}{4 \pi}} = 0.28 \\
  y_1^{-1} & = - \sqrt{\dfrac{3}{4 \pi}} \dfrac{y}{r} = -0.49 \dfrac{y}{r} \\
  y_1^{0} & = \sqrt{\dfrac{3}{4 \pi}} \dfrac{z}{r} = 0.49 \dfrac{z}{r} \\
  y_1^{1} & = - \sqrt{\dfrac{3}{4 \pi}} \dfrac{x}{r} = -0.49 \dfrac{x}{r}
  \end{aligned}
  \right.
  $$

- 为什么球谐函数能更好地表达颜色？
  - 直觉上：参数数量多（RGB 三个 vs C 矩阵 $16 \times 3$ 个），维度高，储存的信息多
  - CG：[环境贴图（Environment Map）](https://learnopengl.com/Advanced-OpenGL/Cubemaps)
    - Skybox，rendering volume 的六个面上存贴图，场景内的反射面上就能反射出四周的环境
    - 越高阶的球谐函数，模拟的 skybox 就越逼真
    - 在渲染中是在用球谐函数来编码亮度和颜色

- 足迹合成
  - 直观上：[Alpha Blending](https://learnopengl.com/Advanced-OpenGL/Blending)
  ```python
  sheets = []
  for g in gaussians:
      sheets.append(g.footprint)
  alpha_blending(sheets)
  ```
  - 实际上：g.footprint，依旧对每个像素进行着色
  ```python
  footprint = np.zeros((H, W, 3))
  for i in range(H):
      for j in range(W):
          footprint[i, j] = ...    
  ```

- 像素的颜色
  - 体渲染：对光线上的粒子颜色进行求和【NeRF玩剩下的】
    - 连续版本：
      - $\displaystyle C = \int_0^{+\infty} T(s) \sigma(s) C(s) \mathrm{d}s = \sum_i T_i \alpha_i C_i$
      - $T(s)$：在 s 点之前，光线没有被阻挡的概率，【由两个已知求出来】
        - $T(s) = \exp(-\int_0^s \sigma(t) \mathrm{d}t)$，这是按概率论列微分方程解出来的解析解
      - $\sigma(s)$：在 s 点处，光线碰撞粒子（光线被粒子阻碍）的概率，【已知】
      - $C(s)$：在 s 点处，粒子的颜色，【已知】
    - 离散化：
      - 将光线 $[0, s]$ 划分为 N 个等间距区间 $[T_n, T_{n+1}]$，$n = 0, 1, \cdots, N$
        - 间隔长度为 $\delta_n$
        - 假设区间内的密度 $\sigma_n$ 和颜色 $C_n$ 固定
      - $\displaystyle C(r) = \sum_{n=0}^N C_n e^{- \sum_{i=0}^{n-1} \sigma_i \delta_i} (1 - e^{- \sigma_n \delta_n}) = \sum_n C_n T_n \alpha_n$，其中：
        - $C_n$ 叫颜色，是之前通过球谐函数算出来的 RGB 值
        - $T_n = e^{- \sum_{i=0}^{n-1} \sigma_i \delta_i}$ 叫没有被阻挡的概率
          - $T_n = 0$ 的话直接不用算了，因为被挡住了
        - $\alpha_n = 1 - e^{- \sigma_n \delta_n}$ 叫不透明度
          - $\alpha_i$ 由 Gaussian 椭球自身的不透明度，以及当前点距离椭球的距离衰减组成
  - 和 NeRF 一样的公式，凭什么 3DGS 就快呢？
    - 粒子数量并没有减少，主要快在 GPU 对每个像素并行处理
    - 3DGS**没有**找粒子的过程
      - 需要对高斯椭球按照深度 z 来排序

- 3DGS 渲染流程
```python
out_color = np.zeros((H, W, 3))
pbar = tqdm(range(H * W))

# 并行处理每个 Pixel
for i in range(H):
    for j in range(W):
        pixf = [i, j]
        C = [0, 0, 0]

        # 由近及远遍历 Gaussian
        for idx in point_list:

            # init helper variables, transmirrance
            T = 1

            # Resample using conic matrix
            # (cf. "Surface Splatting" by Zwicker et al., 2001)
            xy = points_xy_image[idx]  # center of 2d gaussian
            d = [
                xy[0] - pixf[0],
                xy[1] - pixf[1],
            ]  # distance from center of pixel
            con_o = conic_opacity[idx]
            power = (
                -0.5 * (con_o[0] * d[0] * d[0] + con_o[2] * d[1] * d[1])
                - con_o[1] * d[0] * d[1]
            )
            if power > 0:
                continue

            # Eq. (2) from 3D Gaussian splatting paper.
            # Compute color
            alpha = min(0.99, con_o[3] * np.exp(power))
            if alpha < 1 / 255:
                continue
            test_T = T * (1 - alpha)
            if test_T < 0.0001:
                break

            # Eq. (3) from 3D Gaussian splatting paper.
            color = features[idx]
            for ch in range(3):
                C[ch] += color[ch] * alpha * T

            T = test_T

        # get final color
        for ch in range(3):
            out_color[j, i, ch] = C[ch] + T * bg_color[ch]

return out_color
```
- 上式中的 `power`：$\displaystyle \dfrac{(x - \mu_1)^2}{{\sigma_1}^2} + \dfrac{(y - \mu_2)^2}{{\sigma_2}^2} + \dfrac{(x - \mu_1)(y - \mu_2)}{\sigma_1 \sigma_2} = \mathrm{constant}$